In [1]:
language = "french"
setting = "Post-apocalypse zombie"
first_message = "Vous vous trouvez dans une communauté isolée, barricadée et surveillée, où quelques dizaines de survivants comme vous ont trouvé refuge. Vous avez perdu des proches lors de l'épidémie et vous êtes déterminé à trouver un moyen de vaincre les Errants et de reconstruire votre monde. Vous avez reçu une mission, et vous savez que votre seul espoir de survie réside dans la découverte d'un remède contre l'épidémie. Vous devez partir à la recherche d'informations sur ce remède, mais pour cela, vous devrez quitter la sécurité relative de votre communauté et affronter les dangers du monde extérieur."

USE_LOCAL_MODEL = True

In [2]:
import os

from dotenv import load_dotenv

from tracet import ChatFireworks, ChatOllama, BaseLLM

load_dotenv()

# These don't work well...
# set_debug(True)
# set_verbose(True)

# Use local model or cloud model ?
USE_LOCAL_MODEL = True
MAX_TOKENS = 8192

model: BaseLLM = None


if USE_LOCAL_MODEL:
    print("Loading local model...")

    model = ChatOllama(
        model="llama3.1:8b",
        temperature=1.0,
        max_tokens=MAX_TOKENS,
    )

    print("Local model loaded.")
else:
    api_key = os.getenv("FIREWORKS_API_KEY")
    # Use Fireworks
    if api_key is None:
        raise ValueError("FIREWORKS_API_KEY not found in environment variables")

    print("Loading Fireworks model...")
    model = ChatFireworks(
        model="accounts/fireworks/models/llama-v3-70b-instruct",
        # model="accounts/fireworks/models/llama-v3p1-8b-instruct",
        api_key=api_key,
        temperature=1.0,
        max_tokens=MAX_TOKENS,
    )
    print("Fireworks model loaded.")

/Users/frieder/Documents/GitHub/procedural-stories copy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading local model...
Local model loaded.


In [3]:
from owlready2 import *

onto = get_ontology("file://story_poptest70b.rdf").load()

# Verify that the story world is internally coherent, at least by OWL standards
# with onto:
# sync_reasoner()

# for l in onto.CurrentLocation.instances(): print(f"CurrentLocation: {l.hasName}")
# for c in onto.Character.instances(): print(c.hasName)

# print(onto.Player.instances()[0].INDIRECT_isLocatedAt)

Issue: The reasoner will use the state of the ontology at the moment it is instantiated.
If the player moves to a new location, the reasoner will not be updated with this new information.

Source: https://dice-group.github.io/owlapy/usage/reasoning_details.html

Solution: Use isolated worlds, one that is clean, which we update and change with asserted facts, the second one that is used for inference.
When the player moves, we change the clean world, and use it for inference.


In [4]:
from generator.utils import (
    encode_entity_name,
    decode_entity_name,
    find_levenshtein_match,
)

# Conversation loop
So, we have pre-generated the first AI message, which lays out the start of the game.

Then, all subsequent turns will use the same system prompt, as well as dynamically enhancing the player's message with relevant information from the KG

After the player sends their message, we either can:

- Have the LLM respond without knowledge, then analyze the message pair for changes to the KG
- Have the LLM analyze the user's message for intent (especially moving to new locations, which will require the information of the new location to generate the response)

I think the second approach is just better, especially in the case of moving to new locations, which change the knowledge necessary to generate the response.



Once the AI response is generated, we also want to extract changes to the world from the message pair, based on the AI response (item lost, item gained)


So... Basically we need to do intent analysis of the Human message.


Let's summarize a typical conversation turn:

0. The player reads the game's message and decides on something to do.
1. The player's message is analyzed with a prompt, made to extract actionable intent that requires new knowledge (move to other location)
2. If the player moves to another location, we need to retrieve the information about this new location and use that info to generate the AI response. At the same time, we update the KG (player's position)
3. Generate the game response
4. While the player reads and decides on their next turn and types it, analyze the last (message, response) pair for changes to the world: updates to the player's inventory (remove, add or change item's description), updates to characters.

## Move intent
Immediately after the player's message, we need to know if we need to load information about a different location, in order to guide the generation of the response.

In [5]:
INTENT_ANALYSIS_TEMPLATE = """
Human: Please analyze the player's message and determine if they **explicitly** intend to move to a nearby location **immediately**.
Nearby locations: {{nearby_locations_names}}

Previous game response: "{{previous_game_response}}"
Player message: "{{message}}"

**Please output only the *exact location name* or 'none' with ABSOLUTELY no additional explanation, notes or details. If it doesn't fit, output 'none'**
Output:
""".strip()

In [6]:
from tracet import Chain, chainable


@chainable(input_keys=["onto"], output_key="nearby_locations_names")
def get_nearby_locations(onto):
    with onto:
        player = onto.Player.instances()[0]
        current_location = player.INDIRECT_isLocatedAt
        nearby_locations = current_location.INDIRECT_isLinkedToLocation

        nearby_locations_names = []
        for location in nearby_locations:
            if location.name == "CurrentLocation":
                continue

            nearby_locations_names.append(location.hasName)

        nearby_locations = ", ".join(f'"{name}"' for name in nearby_locations_names)

        return nearby_locations


@chainable(
    input_keys=["move_intent_response", "onto"], output_key="move_intent_location"
)
def find_location(move_intent_response: str, onto):
    result = move_intent_response.strip().strip('"')
    if result.lower() == "none":
        return None
    else:
        entity = find_levenshtein_match(result, onto.Location.instances())
        return entity


move_intent_extraction_chain = Chain(
    get_nearby_locations,
    INTENT_ANALYSIS_TEMPLATE,
    model.using(
        output_key="move_intent_response",
        temperature=0.0,
    ),
    find_location,
    verbose=True,
)


def extract_move_intent(model, message, onto, previous_game_response=first_message):
    return move_intent_extraction_chain.call(
        message=message, onto=onto, previous_game_response=previous_game_response
    )

In [7]:
print(extract_move_intent(model, "Aller à la mine", onto))
print(extract_move_intent(model, "Aller au bunker", onto))
print(extract_move_intent(model, "Manger une pomme", onto))
print(extract_move_intent(model, "Aller au marché", onto))

get_nearby_locations output (key: nearby_locations_names):
"Le Marché Noir", "La Route Perdue"

Jinja2Template output (key: prompt):
Human: Please analyze the player's message and determine if they **explicitly** intend to move to a nearby location **immediately**.
Nearby locations: &#34;Le Marché Noir&#34;, &#34;La Route Perdue&#34;

Previous game response: "Vous vous trouvez dans une communauté isolée, barricadée et surveillée, où quelques dizaines de survivants comme vous ont trouvé refuge. Vous avez perdu des proches lors de l&#39;épidémie et vous êtes déterminé à trouver un moyen de vaincre les Errants et de reconstruire votre monde. Vous avez reçu une mission, et vous savez que votre seul espoir de survie réside dans la découverte d&#39;un remède contre l&#39;épidémie. Vous devez partir à la recherche d&#39;informations sur ce remède, mais pour cela, vous devrez quitter la sécurité relative de votre communauté et affronter les dangers du monde extérieur."
Player message: "Aller à

## Response generation

In [8]:
CHAT_SYSTEM_PROMPT = """
You are an LLM designed to act as the engine for a text adventure game set in "{{setting}}".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large. You should discreetly tell the player they can go there, without being too explicit.
If the player has spent a long time in a location, you can push them a little more explicitly to move to different locations.
The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.
**Do not reveal character names unless the player knows them. Do not reveal the detailed character descriptions unless the player asks for them.**

Your messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can always ask for more details ! For dialogues, you will output only one line of dialogue, and let the player respond.
Try to move the story forward towards the player's goal, which unless stated in the game state, is probably in another location.

Use the game elements provided:

# Game state
The player's current location is "{{info.current_location.hasName}}". {{info.current_location.hasName}} is described as "{{info.current_location.hasDescription}}".

The locations accessible from where the player is are {{info.nearby_locations_names}}. You should discreetly tell the player they can go there, without being too explicit.

The player's character is named "{{info.player.hasName}}" and is described as "{{info.player.hasDescription}}".
The player's goal is "{{info.player.hasGoal[0].hasDescription}}"

{%- if info.characters_nearby %}
Characters present in {{info.current_location.hasName}}:
{%- for character in info.characters_nearby %}
    - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
{%- endfor %}
{%- else %}
    There are no other characters present in {{info.current_location.hasName}}.
{%- endif %}

{%- if info.items_nearby %}
Items present in {{info.current_location.hasName}}:
{%- for item in info.items_nearby %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    There are no items present in {{info.current_location.hasName}}.
{%- endif %}

{%- if info.player.INDIRECT_ownsItem %}
Items in the player's inventory:
{%- for item in info.player.INDIRECT_ownsItem %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    The player has no items in their inventory.
{%- endif %}

{%- if move_intent_location %}
The parser has detected that the player intends to move to {{move_intent_location.hasName}}.
    {{move_intent_location.hasName}} is described as "{{move_intent_location.hasDescription}}".
    {%- if move_intent_location.INDIRECT_containsCharacter %}
        Characters present in {{move_intent_location.hasName}}:
        {%- for character in move_intent_location.INDIRECT_containsCharacter %}
            - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
        {%- endfor %}
    {%- endif %}
    {%- if move_intent_location.INDIRECT_containsItem %}
        Items present in {{move_intent_location.hasName}}:
        {%- for item in move_intent_location.INDIRECT_containsItem %}
            - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
        {%- endfor %}
    {%- endif %}

If you confirm the move, add the token "<CONFIRM_MOVE>" to your response.
**Keep in mind that the player will read your response before typing theirs, and only the token will be removed from your response.**
{%- endif %}

Always answer in {{language}}
""".strip()

In [9]:
@chainable(input_keys=["history"], output_key="previous_game_response")
def get_previous_game_response(history):
    return history[-1]["content"]


@chainable(
    input_keys=["move_intent_location", "history", "message", "onto"],
    output_key="info",
)
def get_relevant_information(onto):
    # Retrieve relevant information from the KG
    with onto:
        player = list(onto.Player.instances())[0]
        current_location = player.INDIRECT_isLocatedAt
        characters_nearby = current_location.INDIRECT_containsCharacter
        locations_nearby = current_location.INDIRECT_isLinkedToLocation
        nearby_locations_names = [location.hasName for location in locations_nearby]
        items_nearby = current_location.INDIRECT_containsItem

        return {
            "current_location": current_location,
            "characters_nearby": characters_nearby,
            "locations_nearby": locations_nearby,
            "nearby_locations_names": nearby_locations_names,
            "items_nearby": items_nearby,
            "player": player,
        }


@chainable(
    input_keys=["game_response", "move_intent_location", "onto"],
    output_key="cleaned_game_response",
)
def extract_move_confirmation(game_response: str, move_intent_location, onto):
    # Find if the LLM accepted the move, and filter out the <CONFIRM_MOVE> token
    cleaned_game_response = game_response
    if "<CONFIRM_MOVE>" in game_response:
        print("Move confirmed by the LLM")
        cleaned_game_response = cleaned_game_response.replace(
            "<CONFIRM_MOVE>", ""
        ).strip()

        # If the LLM accepted the move, update the KG
        with onto:
            player = onto.Player.instances()[0]
            for follower in player.INDIRECT_hasFollower:
                follower.isLocatedAt = move_intent_location
            player.isLocatedAt = move_intent_location

    # Overwrite the response with the new one
    return cleaned_game_response

### Create a conversation history

In [10]:
from tracet import ConversationMemory, Jinja2Template

conversation_chain = Chain(
    get_previous_game_response,
    move_intent_extraction_chain,
    get_relevant_information,
    Jinja2Template(  # Format system prompt with the relevant information
        CHAT_SYSTEM_PROMPT,
        output_key="system_prompt",
    ),
    model.using(
        output_key="game_response",
        history_key="history",
        input_key="message",
        system_prompt_key="system_prompt",
        temperature=1.0,
    ),
    extract_move_confirmation,
    ConversationMemory(
        human_message_key="message",
        llm_response_key="cleaned_game_response",
        output_key="history",
        human_prefix="player",
        llm_prefix="game",
    ),
    verbose=True,
)

# Test the conversation chain

In [11]:
response = conversation_chain.call(
    message="Aller à la mine",
    history=[{"role": "game", "content": first_message}],
    onto=onto,
    setting=setting,
    language=language,
)

get_previous_game_response output (key: previous_game_response):
Vous vous trouvez dans une communauté isolée, barricadée et surveillée, où quelques dizaines de survivants comme vous ont trouvé refuge. Vous avez perdu des proches lors de l'épidémie et vous êtes déterminé à trouver un moyen de vaincre les Errants et de reconstruire votre monde. Vous avez reçu une mission, et vous savez que votre seul espoir de survie réside dans la découverte d'un remède contre l'épidémie. Vous devez partir à la recherche d'informations sur ce remède, mais pour cela, vous devrez quitter la sécurité relative de votre communauté et affronter les dangers du monde extérieur.

get_nearby_locations output (key: nearby_locations_names):
"Le Marché Noir", "La Route Perdue"

Jinja2Template output (key: prompt):
Human: Please analyze the player's message and determine if they **explicitly** intend to move to a nearby location **immediately**.
Nearby locations: &#34;Le Marché Noir&#34;, &#34;La Route Perdue&#34;



In [12]:
print(response["cleaned_game_response"])

La mine ? Vous voulez aller à la mine. Mais je pense que vous devriez plutôt réfléchir... La commotion des événements du marché noir vous a peut-être donné une idée de ce qui se passe vraiment. La Route Perdue pourrait être votre meilleure option pour obtenir des informations précieuses.

Vous pouvez aller à Le Marché Noir, si vous le souhaitez. Raphaël Boucher a l'air d'être en train de parler avec Léa Morin, et il est peut-être temps pour vous d'obtenir plus d'informations.


# Post processing

After the response, we need to report any change to the world state into the knowledge graph.

A player needs to be able to interact with the graph in a few ways:
- move to a nearby location (handled in pre-response generation step)
- claim ownership (of an unowned item): take, buy
- change ownership (of an owned item): give, drop, sell
- change in emotional state of any character
- change in relationship between characters
- add or remove a follower of the player
- update item description (for example if an item deteriorates, etc)
- update character description (for example if a character needs to remember something)
- update location description (for example if the player changed the location in some notable way)
- trigger event
- update events memory (?)
- death of a character (?)

But these are a lot of tasks to ask from one prompt to a small model like Llama3.1-8B-8bit

We can classify these tasks in groups:
- Inventory related tasks
- Character related tasks
- Location related tasks

Now, we can imagine using a prompt for each one of these categories, in order to get a fair balance of speed and accuracy

In [13]:
INVENTORY_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and determine if they intend to interact with an item.

# Action Types
- "create" if you need to create a new item that doesn't exist yet.
- "destroy" if an item should be removed from the game.
- "claim" if the player has taken or picked up an unowned item.
- "give" if the player has given an item they own to another character. In this case, you will provide the name of the character the player gives the item to. **If the character doesn't exist, ignore the action.**
- "drop" if the player has dropped an item they own in the current location.
- "alter" if the player's actions have changed the state of an item. In this case, you will provide an updated description of the item.

{%- if player.INDIRECT_ownsItem %}
# Possible items whose state to change
**Owned items**:
{%- for item in player.INDIRECT_ownsItem %}
- {{item.hasName}}: {{item.hasDescription}}
{%- endfor %}
{%- else %}
There are no items in the player's inventory.
{%- endif %}

{%- if current_location.INDIRECT_containsItem %}
**Unowned items in the current location**:
{%- for item in current_location.INDIRECT_containsItem %}
{%- if not item.INDIRECT_isOwnedBy %}
- "{{item.hasName}}": {{item.hasDescription}}
{%- endif %}
{%- endfor %}
{%- else %}
There are no unowned items in the current location.
{%- endif %}

{%- if current_location.INDIRECT_containsCharacter %}
**Names of the characters present in the current location**:
{%- for character in current_location.INDIRECT_containsCharacter %}
    {%- if character.hasName != player.hasName %}
- "{{character.hasName}}"
    {%- endif %}
{%- endfor %}
{%- else %}
There are no other characters present in the current location.
{%- endif %}

# Last message from the player and game response
Player message: "{{message}}"
Game response: "{{game_response}}"

**Please output a list of actions that the game system needs to perform to update the game state, in the following JSON format:**
If none of these fit, output an empty list.

```json
{
    "actions": [
        {"action": "create", "item": "map", "description": "A map of the area."},
        {"action": "claim", "item": "map"},
        {"action": "give", "item": "coin", "target": "John Brown"},
        {"action": "drop", "item": "bucket"},
        {"action": "destroy", "item": "apple"},
        {"action": "alter", "item": "sword", "description": "The sword is now covered in rust."}
    ]
}
```

"claim" (for the player) and "give" (to the player) are always from the player's point of view.
**In order to give an item that doesn't exist yet in the game system to the player, you need to create it first with a "create" action, before you can claim it with a "claim" action.**
ALWAYS USE THE SAME NAME FOR THE ITEM IN THE JSON OUTPUT AS THE ONE IN THE GAME STATE (IN QUOTES). Otherwise, the game will not be able to find the item.

Output:
""".strip()

In [14]:
@chainable(input_keys=["onto"], output_key="current_location")
def get_current_location(onto):
    with onto:
        player = onto.Player.instances()[0]
        current_location = player.INDIRECT_isLocatedAt

    return current_location


@chainable(input_keys=["onto"], output_key="player")
def get_player(onto):
    with onto:
        return onto.Player.instances()[0]

In [15]:
from tracet import JsonRepairParser

inventory_actions_chain = Chain(
    get_current_location,
    get_player,
    INVENTORY_ACTIONS_PARSER_TEMPLATE,
    model,
    JsonRepairParser(output_key="inventory_actions"),
    verbose=True,
    debug=True,
)

In [16]:
# Test the chain
inventory_actions = inventory_actions_chain.call(
    onto=onto,
    message="Je prends la carte",
    game_response="Vous prenez la carte.",
)
print(inventory_actions)

Chain started with state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Je prends la carte",
  "game_response": "Vous prenez la carte."
}

get_current_location output (key: current_location):
story_poptest70b.la%20communaut%C3%A9

Chain state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Je prends la carte",
  "game_response": "Vous prenez la carte.",
  "current_location": "story_poptest70b.la%20communaut%C3%A9"
}

get_player output (key: player):
story_poptest70b.alexandre%20dumont

Chain state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Je prends la carte",
  "game_response": "Vous prenez la carte.",
  "current_location": "story_poptest70b.la%20communaut%C3%A9",
  "player": "story_poptest70b.alexandre%20dumont"
}

Jinja2Template output (key: prompt):
Human: Please analyze the player's message and determine if they intend to interact

In [17]:
@chainable(input_keys=["inventory_actions", "onto"], output_key=None)
def apply_inventory_actions(inventory_actions: list, onto):
    if not inventory_actions:
        print("No inventory actions to apply.")
        return
    actions = inventory_actions.get("actions", [])
    with onto:
        player = onto.Player.instances()[0]
        for action in actions:
            try:
                if action["action"] == "create":
                    item_name = action["item"]
                    item = onto.Item(encode_entity_name(item_name))
                    item.hasDescription = action["description"]
                    player.ownsItem.append(item)
                elif action["action"] == "destroy":
                    item = find_levenshtein_match(action["item"], onto.Item.instances())
                    if item:
                        player.ownsItem.remove(item)
                        destroy_entity(item)
                    else:
                        print(f"Item not found: {action['item']}")
                elif action["action"] == "claim":
                    item = find_levenshtein_match(action["item"], onto.Item.instances())
                    if item:
                        player.ownsItem.append(item)
                        item.itemIsLocatedAt = None
                    else:
                        print(f"Item not found: {action['item']}")
                elif action["action"] == "give":
                    item = find_levenshtein_match(action["item"], onto.Item.instances())
                    target = find_levenshtein_match(
                        action["target"], onto.Character.instances()
                    )
                    if item and target:
                        if item in player.ownsItem:
                            player.ownsItem.remove(item)
                            target.ownsItem.append(item)
                    else:
                        if not item:
                            print(f"Item not found: {action['item']}")
                        if not target:
                            print(f"Target not found: {action['target']}")
                elif action["action"] == "drop":
                    item = find_levenshtein_match(action["item"], onto.Item.instances())
                    if item:
                        player.ownsItem.remove(item)
                        current_location = get_current_location(onto)
                        item.itemIsLocatedAt = current_location
                    else:
                        print(f"Item not found: {action['item']}")
                elif action["action"] == "alter":
                    item = find_levenshtein_match(action["item"], onto.Item.instances())
                    if item:
                        description = action.get("description", "")
                        if description:
                            item.hasDescription = description
                        else:
                            print(f"No description provided for item: {action['item']}")
                    else:
                        print(f"Item not found: {action['item']}")
            except Exception as e:
                print(f"Error applying inventory action: {e}")

In [18]:
CHARACTER_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and corresponding game response to determine if the interaction changed something about the characters present in the scene

**Action Types**:.
- "start_following" if a character has started following the player.
- "stop_following" if a character has stopped following the player.
- "change_health" if a character's health has changed. Add a new description of the character's health.
- "change_description" if a character's description needs to be changed because something **important** has changed about the character. Only include information about the nature of the character, not what they are doing or where they are. Add a new description of the character. **DO NOT INCLUDE LOCATION OR INVENTORY INFORMATION, OR OTHER TEMPORARY INFORMATION. ONLY INCLUDE INFORMATION THAT IS IMPORTANT AND SHOULD BE REMEMBERED. MOST INFORMATION SHOULDN'T BE REMEMBERED.**
- "become_enemies" if two characters have become enemies. (symmetrical)
- "become_friends" if two characters have become friends. (symmetrical)
- "become_rivals" if two characters have become rivals. (symmetrical)
- "become_neutral" if two characters have become neutral. (symmetrical)
- "give_allegiance" if a character has given allegiance to another character. (asymmetrical)
- "now_knows" if a character now knows about another character's existence. (asymmetrical)
- "fall_in_love" if a character has fallen in love with another character. (asymmetrical)

Most relationships are symmetrical, but some are not, like loves. If love is reciprocal, output two actions, one for each character.

The player is named "{{player.hasName}}" and is described as "{{player.hasDescription}}".

{%- if current_location.INDIRECT_containsCharacter %}
**Characters present**:
{%- for character in current_location.INDIRECT_containsCharacter %}
- {{character.hasName}}: {{character.hasDescription}}
    {% if character.INDIRECT_knows %}knows: {%- for known_character in character.INDIRECT_knows %} "{{known_character.hasName}}" {%- endfor %}{%- endif %}
    {% if character.INDIRECT_isEnemyWith %}is enemy with: {%- for enemy in character.INDIRECT_isEnemyWith %} "{{enemy.hasName}}" {%- endfor %}{%- endif %}
    {% if character.INDIRECT_hasFriendshipWith %}is friend with: {%- for friend in character.INDIRECT_hasFriendshipWith %} "{{friend.hasName}}" {%- endfor %}{%- endif %}
    {% if character.INDIRECT_hasRivalryWith %}is rival with: {%- for rival in character.INDIRECT_hasRivalryWith %} "{{rival.hasName}}" {%- endfor %}{%- endif %}
    {% if character.INDIRECT_loves %}is in love with: {%- for love in character.INDIRECT_loves %} "{{love.hasName}}" {%- endfor %}{%- endif %}
    {% if character.INDIRECT_hasAllegiance %}has allegiance to: {{character.INDIRECT_hasAllegiance.hasName}}{%- endif %}
{%- endfor %}
{%- else %}
There are no characters present in the current location.
{%- endif %}

Player message: "{{message}}"
Game response: "{{game_response}}"

**Please output a list of actions that the game system needs to perform to update the game state, in the following JSON format:**
**If none of these fit, output an empty list.**

```json
{
    "actions": [
        {"action": "become_enemies", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "become_friends", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "become_neutral", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "fall_in_love", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "give_allegiance", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "now_knows", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "start_following", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "stop_following", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "change_health", "subject": "John Brown", "description": "John Brown is in dire condition. He might die soon."},
        {"action": "change_description", "subject": "John Brown", "description": "... (**keep most of the original description, only add or change things that are important**) and is now wearing a red hat."}
    ]
}
```

Output:
""".strip()

In [19]:
character_actions_chain = Chain(
    get_current_location,
    get_player,
    CHARACTER_ACTIONS_PARSER_TEMPLATE,
    model.using(
        output_key="character_actions_response",
        temperature=0.0,
    ),
    JsonRepairParser(
        input_key="character_actions_response", output_key="character_actions"
    ),
    verbose=True,
    debug=True,
)

In [20]:
character_actions = character_actions_chain.call(
    onto=onto,
    message="Suis-moi, Léa",
    game_response="Léa vous suit désormais. Vous vous dirigez vers la mine.",
)
print(character_actions)

Chain started with state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Suis-moi, L\u00e9a",
  "game_response": "L\u00e9a vous suit d\u00e9sormais. Vous vous dirigez vers la mine."
}

get_current_location output (key: current_location):
story_poptest70b.la%20communaut%C3%A9

Chain state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Suis-moi, L\u00e9a",
  "game_response": "L\u00e9a vous suit d\u00e9sormais. Vous vous dirigez vers la mine.",
  "current_location": "story_poptest70b.la%20communaut%C3%A9"
}

get_player output (key: player):
story_poptest70b.alexandre%20dumont

Chain state:
{
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "message": "Suis-moi, L\u00e9a",
  "game_response": "L\u00e9a vous suit d\u00e9sormais. Vous vous dirigez vers la mine.",
  "current_location": "story_poptest70b.la%20communaut%C3%A9",
  "player": "story_poptest70b.alexandr

In [21]:
@chainable(input_keys=["character_actions", "onto"], output_key=None)
def apply_character_actions(character_actions: list, onto):
    if not character_actions:
        print("No character actions to apply")
        return
    actions = character_actions.get("actions", [])
    with onto:
        player = onto.Player.instances()[0]
        for action in actions:
            try:
                if action["action"] == "start_following":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    if subject:
                        player.hasFollower.append(subject)
                elif action["action"] == "stop_following":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    if subject:
                        if subject in player.hasFollower:
                            player.hasFollower.remove(subject)
                    else:
                        print(f"Subject not found: {action['subject']}")
                elif action["action"] == "change_health":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    if subject:
                        subject.hasHealth = action["description"]
                    else:
                        print(f"Subject not found: {action['subject']}")
                elif action["action"] == "change_description":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    if subject:
                        subject.hasDescription = action["description"]
                    else:
                        print(f"Subject not found: {action['subject']}")
                elif action["action"] == "become_enemies":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        subject.isEnemyWith.append(_object)
                        _object.isEnemyWith.append(subject)
                    else:
                        if not subject:
                            print(f"Subject not found: {action['subject']}")
                        if not _object:
                            print(f"Object not found: {action['object']}")
                elif action["action"] == "become_friends":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        subject.hasFriendshipWith.append(_object)
                        _object.hasFriendshipWith.append(subject)
                    else:
                        if not subject:
                            print(f"Subject not found: {action['subject']}")
                        if not _object:
                            print(f"Object not found: {action['object']}")
                elif action["action"] == "become_neutral":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        if _object in subject.isEnemyWith:
                            subject.isEnemyWith.remove(_object)
                        if _object in subject.hasRivalryWith:
                            subject.hasRivalryWith.remove(_object)
                        if _object in subject.hasFriendshipWith:
                            subject.hasFriendshipWith.remove(_object)
                        if _object in _object.isEnemyWith:
                            _object.isEnemyWith.remove(subject)
                        if _object in _object.hasRivalryWith:
                            _object.hasRivalryWith.remove(subject)
                        if _object in _object.hasFriendshipWith:
                            _object.hasFriendshipWith.remove(subject)
                elif action["action"] == "fall_in_love":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        # Love is sometimes reciprocal, sometimes not
                        subject.loves.append(_object)
                    else:
                        if not subject:
                            print(f"Subject not found: {action['subject']}")
                        if not _object:
                            print(f"Object not found: {action['object']}")
                elif action["action"] == "give_allegiance":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        subject.hasAllegiance = _object
                    else:
                        if not subject:
                            print(f"Subject not found: {action['subject']}")
                        if not _object:
                            print(f"Object not found: {action['object']}")
                elif action["action"] == "rescind_allegiance":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    if subject:
                        subject.hasAllegiance = None
                    else:
                        print(f"Subject not found: {action['subject']}")
                elif action["action"] == "now_knows":
                    subject = find_levenshtein_match(
                        action["subject"], onto.Character.instances()
                    )
                    _object = find_levenshtein_match(
                        action["object"], onto.Character.instances()
                    )
                    if subject and _object:
                        subject.knows.append(_object)
                    else:
                        if not subject:
                            print(f"Subject not found: {action['subject']}")
                        if not _object:
                            print(f"Object not found: {action['object']}")
            except Exception as e:
                print(f"Error applying character action: {e}")

# Play !

In [22]:
history = []

# Add the first messages to the history
history.append({"role": "game", "content": first_message})

conversation_chain.verbose = False
move_intent_extraction_chain.verbose = False
move_intent_extraction_chain.debug = False
inventory_actions_chain.verbose = False
inventory_actions_chain.debug = False
character_actions_chain.verbose = False
character_actions_chain.debug = False

postprocess_chain = Chain(
    inventory_actions_chain,
    apply_inventory_actions,
    character_actions_chain,
    apply_character_actions,
)


async def converse(message: str):
    response = await conversation_chain.acall(
        message=message,
        history=history,
        onto=onto,
        setting=setting,
        language=language,
    )
    return response["cleaned_game_response"]


while True:
    message = input("Player: ")

    if message.lower() in ("exit", "quit"):
        print("Thanks for playing!")
        break

    print(f"Player: {message}")

    game_response = await converse(message)
    print(f"Game: {game_response}")

    postprocess_chain.call(
        message=message,
        game_response=game_response,
        onto=onto,
    )

Player: J'inspecte mon inventaire
Game: Vous vérifiez vos affaires et vous constatez que vous avez encore un couteau de chasse, des rations de survie et une bouteille d'eau. La carte de la communauté est encore là, mais vous ne voyez aucune autre ressource nouvelle dans votre inventaire.

La Communauté reste calme, les gardes semblent être là où ils devraient l'être. Vous remarquez que Léa Morin est dans un coin, parlant à voix basse avec Raphaël Boucher.


ChainExecutionError: Error in component 'apply_inventory_actions'

[1mChain state before failure:[0m
{
  "message": "J'inspecte mon inventaire",
  "game_response": "Vous v\u00e9rifiez vos affaires et vous constatez que vous avez encore un couteau de chasse, des rations de survie et une bouteille d'eau. La carte de la communaut\u00e9 est encore l\u00e0, mais vous ne voyez aucune autre ressource nouvelle dans votre inventaire.\n\nLa Communaut\u00e9 reste calme, les gardes semblent \u00eatre l\u00e0 o\u00f9 ils devraient l'\u00eatre. Vous remarquez que L\u00e9a Morin est dans un coin, parlant \u00e0 voix basse avec Rapha\u00ebl Boucher.",
  "onto": "get_ontology(\"http://www.florianrieder.com/semantic/story.owl#\")",
  "current_location": "story_poptest70b.la%20communaut%C3%A9",
  "player": "story_poptest70b.alexandre%20dumont",
  "prompt": "Human: Please analyze the player's message and determine if they intend to interact with an item.\n\n# Action Types\n- \"create\" if you need to cr